In [1]:
import pandas as pd
import numpy as np
import requests
import ast 
import datetime

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [ ]:
paises = {"USA" : [39.7837304,-100.445882], "AUSTRALIA" : [-24.7761086, 134.755], "SOUTH AFRICA" : 
    [-28.8166236, 24.991639], "NEW ZEALAND" : [-41.5000831, 172.8344077], "PAPUA NEW GUINEA" : [-5.6816069, 144.2489081]}
producto = "civil"
producto2 = "astro"


In [ ]:
class Extraccion: 
    # primero tenemos que crear el constructor con las variables globales que usaremos. Están son variables que podrán cambiar cada vez que llamemos a la clase. 
    def __init__(self, paises):

        # definimos cada una de las variables. Recordamos que tenemos que usar el método .self para definirlas y que la clase entienda que las podremos usar en otros métodos de la clase. 

        self.paises = paises

    # definimos nuestro primer método. El que nos hará la llamada a la API
    # Si comparamos, veremos que la función es exactamente igual a la que hicimos en la clase anterior, solo que con unos pequeños cambios. En este caso incluimos el parámetro self
    def llamada_API(self, producto):

        self.producto = producto

        df_final = pd.DataFrame()

        for key, value in paises.items():
        
            url = f'http://www.7timer.info/bin/api.pl?lon=-{value[1]}&lat={value[0]}&product={producto}&output=json'

            response = requests.get(url=url)
            codigo_estado = response.status_code
            razon_estado = response.reason
            if codigo_estado == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            elif codigo_estado == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
            
            df = pd.json_normalize(response.json()["dataseries"])
            df['latitud'] = value[0]
            df['longitud'] = value[1]
            df['pais'] = key
            df_final = pd.concat([df_final,df], axis=0, ignore_index=True) 
         
        
    
    # definimos un nuevo método, el de limpiar los datos obtenidos de la llamada a la API para el producto civil. 
    def limpiar_columnas_meteo(self, df_meteo):

        self.df = df_meteo

        df_meteo['wind_profile']= df_meteo['wind_profile'].apply(ast.literal_eval)

        x = df_meteo['wind_profile'].apply(pd.Series)

        for i in range(len(x.columns)): 

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = 'windspeed_' + str(x[i].apply(pd.Series)['layer'][0])

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["speed"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df.insert(i, nombre, valores)

        df_meteo['rh_profile']= df_meteo['rh_profile'].apply(ast.literal_eval)

        x = df_meteo['rh_profile'].apply(pd.Series)

        for i in range(len(x.columns)): 

        # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = 'rh_' + str(x[i].apply(pd.Series)['layer'][0] )
            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(x[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_meteo.insert(i, nombre, valores)

        return df_meteo

    def meteo_medias(self, df_meteo):

        df_meteo_medias = df_meteo.groupby('pais').apply(lambda x: x.mean())

        df_meteo_medias.reset_index(inplace=True)

        df_meteo_medias['pais'] = df_meteo_medias['pais'].apply(lambda x: x.lower())
        
        return df_meteo_medias 


    def juntar_dfs(self, df_meteo_medias, df_ataques): 

        df_ataques_meteo =  pd.merge(left = df_ataques, right= df_meteo_medias, how= "left", left_on= "country", right_on= "pais")

        # guardamos los dato
        df_ataques_meteo.to_pickle('datos/datos_actualizados.pkl')
        df_ataques_meteo.to_csv('datos/datos_actualizados.csv')

        return df_ataques_meteo
    

    def chequear_datos(self, df_ataques_meteo): 
    
        print("Las columnas son:", "\n")
        print(list(df_ataques_meteo.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df_ataques_meteo.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df_ataques_meteo.isnull().sum() / df_ataques_meteo.shape[0]) *  100)
